In [9]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from IPython.display import display
from statistics import mean
import numpy as np

crypto_to_index = {'btt': 75, 'btc': 36, 'etc': 100, 'doge': 87, 'link': 78, 'xlm': 82, 'ltc': 38, 'ada': 80, 'eth': 41, 'matic': 72, 'bnb': 74, 'cake': 114, 'aave': 286, 'shib':125}
cryptos = ['BTT','BTC','ETC','DOGE', 'LINK', 'XLM', "LTC", "ADA", "ETH","MATIC"]
bought_prices = [0.5912, 4251565.282, 3506.84,32.498, 3104.161, 40.46, 29496.96, 137.139, 258716.55, 96.37]
bought_qty = [1967+405+445+454+532,0.0002 + 0.00006 + 0.00011+0.00008,0.33+0.03+0.03, 7+7, 0.3 + 0.04 + 0.07 + 0.09 ,12.1+6.1, 0.003+0.011 , 3.5+2.4 + 1.5 + 5.1,0.0012+0.0011+0.0013,2+5+2+22]

invested_amt = [round(bought_prices[i] * bought_qty[i], 5) for i in range(len(cryptos))]

sold_coin = ["doge", "doge", "usdt", "usdt"]
sold_profit_per_coin = [40.2-35.996, 38-35.996, 80.31-78.32, 80.15-78.32]
sold_qty = [25, 18, 4, 2.17]
realized_profits = [sold_profit_per_coin[i] * sold_qty[i] for i in range(len(sold_qty))]
 
 
def profit_calc(curr_price, bought_price, bought_qty):
  profit_val = (curr_price - bought_price) * bought_qty
  return round(profit_val, 4)
 
def profit_perc_calc(profit_val, bought_price, bought_qty):
  profit_perc = round((profit_val / (bought_price * bought_qty)), 4) * 100
  return profit_perc
 
def fetch_data():
  req = requests.get("https://api.wazirx.com/api/v2/market-status", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
  return req
 
def get_index(crypto):
  req = fetch_data()
  data = json.loads(req.content)
  markets = data['markets']
  for market in markets:
    if market['baseMarket']==crypto and market['quoteMarket']=="inr" and market['type']=="SPOT":
      return markets.index(market)
 
def get_index_aave():
  req = fetch_data()
  data = json.loads(req.content)
  markets = data['markets']
  for market in markets:
    if market['baseMarket']=="aave" and market['quoteMarket']=="usdt" and market['type']=="SPOT":
      return markets.index(market)

def get_all_indexes():
  crypto_indexes = {}
  for crypto in cryptos:
    if crypto.lower()!="aave":
      crypto_indexes[crypto.lower()] = get_index(crypto.lower())
    else:
      crypto_indexes[crypto.lower()] = get_index_aave()
  return crypto_indexes

 
 
def add_more_existing_crypto(crypto, new_qty, new_price, bought_prices, bought_qty):
  index_crypto = cryptos.index(crypto.upper())
  final_qty = bought_qty[index_crypto] + new_qty
  final_price = (bought_qty[index_crypto] * bought_prices[index_crypto] + new_qty * new_price) / (final_qty) #perform weighted average for geting average price of one coin
  bought_prices[index_crypto] = final_price
  bought_qty[index_crypto] = final_qty
  return bought_prices, bought_qty
 
def get_live_data(cryptos, req):
  data = json.loads(req.content)
  markets = data['markets']
  aave_ltp = round((float(markets[crypto_to_index["aave"]]['last']) *  float(markets[35]['last'])), 5)
  # print(aave_ltp)
  ltp = []
  for crypto in cryptos:
    indexval = crypto_to_index[crypto.lower()]
    if crypto=="AAVE":
      ltp.append(aave_ltp)
    else:
      ltp.append(round(float(markets[indexval]['last']), 5))
  return ltp
 
def get_price_quant(crypto):
  price_val = bought_prices[cryptos.index(crypto)]
  quant_val = bought_qty[cryptos.index(crypto)]
  print("Average price at cost " + str(price_val))
  print("Quantity " + str(quant_val))
  return price_val, quant_val
 
def color_negative_red(value):
  if value > 0:
    color="lawngreen"
  elif value < 0:
    color = "red" 
  else:
    color = "white"
  return 'color: %s' % color   
 
def get_dataframe():
  req = fetch_data()
  ltp = get_live_data(cryptos, req)
  # print(ltp)
  current_val = [round(bought_qty[i] * ltp[i], 5) for i in range(len(cryptos))]
  profits = [profit_calc(ltp[i], bought_prices[i], bought_qty[i]) for i in range(len(cryptos))]
  profit_percs = [profit_perc_calc(profits[i], bought_prices[i], bought_qty[i]) for i in range(len(cryptos))]
  # print(profit_percs)
  dashboard = {'LTP': ltp,'Profit': profits, 'Invested':invested_amt, "Current value":current_val, 'Profit-Percentage':profit_percs}
  df = pd.DataFrame(dashboard, columns = ['LTP', 'Invested', 'Current value', 'Profit-Percentage', 'Profit'], index=cryptos)
  df.sort_values(by=['Profit-Percentage'], inplace=True, ascending=False)
  styler = (df.style
        .applymap(color_negative_red, subset=['Profit','Profit-Percentage'])
        .set_properties(**{'background-color': 'black','border-color': 'white'})
        .format({'Profit': "{:.2f}"})
        .format({'Profit-Percentage': "{:.2f}%"})
        .format({'LTP': "{:.2f}"})
        .format({'Invested': "{:.2f}"})
        .format({'Current value': "{:.2f}"}))
  return styler, current_val, profits, profit_percs
 
def crypto_dashboard():
  print("----------------------------Real-Time Crypto Dashboard----------------------------------")
  print()
  styler, current_val, profits, profit_percs = get_dataframe()
  display(styler)
  print()
  print()
  print("Total invested amount is " + str(round(sum(invested_amt), 2)))
  print("Total current value is " + str(round(sum(current_val), 2)))
  print("Overall P&L is " + str(round(sum(profits), 2)))
  print("Overall P&L % is " + str(round((sum(profits) / sum(invested_amt)) * 100, 2)) + "%")
  print()
  print()
  print("Realized profit is " + str(round(sum(realized_profits), 2)))

In [10]:
crypto_dashboard()

----------------------------Real-Time Crypto Dashboard----------------------------------



,LTP,Invested,Current value,Profit-Percentage,Profit
MATIC,151.63,2987.47,4700.53,57.34%,1713.06
ETH,330000.00,931.38,1188.00,27.55%,256.62
ETC,4098.00,1367.67,1598.22,16.86%,230.55
ADA,160.00,1714.24,2000.00,16.67%,285.76
BTC,4893204.00,1913.20,2201.94,15.09%,288.74
LINK,2406.28,1552.08,1203.14,-22.48%,-348.94
DOGE,23.31,454.97,326.34,-28.27%,-128.63
XLM,27.10,736.37,493.27,-33.01%,-243.10
LTC,14980.00,412.96,209.72,-49.22%,-203.24
BTT,0.28,2248.33,1067.54,-52.52%,-1180.79




Total invested amount is 14318.67
Total current value is 14988.7
Overall P&L is 670.03
Overall P&L % is 4.68%


Realized profit is 153.1
